In [1]:
import pandas as pd
df_reviews = pd.read_pickle(r'reviewsFromImdb/networkData3.pkl')

In [2]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [3]:
sent_tokens = []
for row in df_reviews["Reviews"]:
    tokens = []
    for sub in row:
        tokens.append(tokenizer.tokenize(sub.lower()))
    sent_tokens.append(tokens)
df_reviews['Review_tokens'] = sent_tokens
        

In [4]:
df_sentimentValues = pd.read_csv('Sentiment_values.txt', sep='\t',engine='python')
df_word_and_scores = df_sentimentValues[['word', 'happiness_average']].copy()
sentiment_dict = dict(zip(df_word_and_scores.word, df_word_and_scores.happiness_average))

In [5]:
ext_list = []
for row in df_reviews['Review_tokens']:
    review_in_list =[]
    for reviews in row:
        tokens_in_list = []
        for token in reviews:
            if token in sentiment_dict.keys():
                tokens_in_list.append(token)
        review_in_list.append(len(tokens_in_list))
    ext_list.append(review_in_list)

In [6]:

reviews_for_movie =[]
for row in df_reviews['Review_tokens']:
    sentiments_for_review = []
    for review in row:
        sentiments_for_token = []
        for token in review:
            if token in sentiment_dict.keys():
                sentiments_for_token.append(sentiment_dict[token])
        sentiments_for_review.append(int(sum(sentiments_for_token)))
    reviews_for_movie.append(sentiments_for_review)
df_reviews['Sentiment'] = reviews_for_movie


In [7]:
del ext_list[1485]
df_reviews.drop(1485,inplace = True)
df_reviews = df_reviews.reset_index(drop=True)

In [8]:
import numpy as np
results = []
for x in range(len(df_reviews['Sentiment'])):
    print(df_reviews['Sentiment'][x])
    results.append((sum(df_reviews['Sentiment'][x])/ len(ext_list[x])))
df_reviews['Avg sentiment movie'] = results

[755, 805, 1575, 1833, 1007, 1620, 1473, 1184, 292, 5356, 1277, 281, 2309, 3029, 356, 634, 1670, 1691, 785, 1067, 992, 303, 195, 1046, 156]
[1387, 1411, 1469, 2889, 910, 1358, 653, 720, 527, 1300, 585, 2145, 436, 809, 590, 1110, 234, 190, 861, 2360, 797, 363, 285, 491, 880]
[2313, 1638, 965, 708, 962, 1521, 954, 358, 856, 232, 389, 733, 656, 84, 409, 547, 1532, 2354, 116, 1063, 537, 928, 2679, 214, 1084]
[1031, 1260, 1444, 302, 515, 267, 1268, 592, 2119, 551, 268, 819, 856, 851, 4234, 1061, 1324, 470, 528, 1450, 1086, 1378, 1756, 2253, 3013]
[702, 865, 680, 599, 4113, 533, 608, 577, 463, 1254, 635, 411, 269, 233, 611, 266]
[742, 1862, 846, 2277, 3148, 783, 2615, 1629, 1630, 179, 1573, 1522, 3060, 1728, 1177, 577, 1358, 1427, 640, 919, 874, 682, 1516, 315, 378]
[576, 759, 2385, 836, 1984, 1257, 469, 827, 588, 874, 1224, 1878, 800, 488, 4704, 868, 838, 632, 979, 696, 2584, 781, 856, 67, 372]
[979, 440, 2382, 833, 2874, 1350, 1646, 440, 1321, 802, 1493, 1110, 2089, 1917, 1553, 1662, 416, 

In [9]:
df_reviews.to_pickle("Sentiment_for_reviews.pkl")
df_reviews

,TitleId,MovieTitle,Actors,Genres,Reviews,Rating,Review_tokens,Sentiment,Avg sentiment movie
0,0235679,Pokémon 3: The Movie,"[Rica Matsumoto, Veronica Taylor, Rachael Lill...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[The other day, I was looking through some old...",5.8,"[[the, other, day, i, was, looking, through, s...","[755, 805, 1575, 1833, 1007, 1620, 1473, 1184,...",1267.640000
1,0221889,Beautiful Creatures,"[Susan Lynch, Iain Glen, Rachel Weisz, Tom Man...","[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...","[What I liked about the film ""Beautiful Creatu...",5.8,"[[what, i, liked, about, the, film, beautiful,...","[1387, 1411, 1469, 2889, 910, 1358, 653, 720, ...",990.400000
2,0268200,Brigham City,"[Richard Dutcher, Wilford Brimley, Carrie Morg...","[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...","[First and foremost, Brigham City is a murder ...",6.7,"[[first, and, foremost, brigham, city, is, a, ...","[2313, 1638, 965, 708, 962, 1521, 954, 358, 85...",953.280000
3,0260332,روزی که زن شدم,"[Fatemeh Cherag Akhar, Hassan Nebhan, Shahr Ba...","[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...","[I had completely forgotten about this movie, ...",7.3,"[[i, had, completely, forgotten, about, this, ...","[1031, 1260, 1444, 302, 515, 267, 1268, 592, 2...",1227.840000
4,0238588,Shadow Magic,"[Jared Harris, Yu Xia, Yufei Xing, Peiqi Liu, ...","[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...",[Ann Hus first directorial and writing effort...,7.2,"[[ann, hu, s, first, directorial, and, writing...","[702, 865, 680, 599, 4113, 533, 608, 577, 463,...",801.187500
...,...,...,...,...,...,...,...,...,...
1990,0066579,Women in Love,"[Alan Bates, Oliver Reed, Glenda Jackson, Jenn...","[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",[Ken Russell's adaptation of D. H. Lawrence's ...,7.1,"[[ken, russell, s, adaptation, of, d, h, lawre...","[1930, 1377, 3407, 2049, 1188, 576, 423, 825, ...",1333.800000
1991,0322824,Japón,"[Alejandro Ferretis, Magdalena Flores, Yolanda...","[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...","[Japon, a film by first-time director Carlos R...",6.9,"[[japon, a, film, by, first, time, director, c...","[2763, 2241, 1864, 493, 1629, 1440, 794, 832, ...",1067.833333
1992,0285462,Boat Trip,"[Horatio Sanz, Roselyn Sanchez, Maurice Godin,...","[{'id': 35, 'name': 'Comedy'}]",[Man I don't understand why people come down s...,4.9,"[[man, i, don, t, understand, why, people, com...","[965, 539, 600, 2531, 312, 961, 641, 772, 5041...",1213.880000
1993,0285531,Dreamcatcher,"[Morgan Freeman, Thomas Jane, Jason Lee, Damia...","[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...","[This Stephen King adaptation is underrated, s...",5.5,"[[this, stephen, king, adaptation, is, underra...","[663, 3448, 1905, 4429, 2080, 555, 1621, 1369,...",1793.280000


In [10]:
print(min(df_reviews['Avg sentiment movie']))
print(max(df_reviews['Avg sentiment movie']))

313.0
2722.44
